## Computational Chemistry for Experimentalists
## Module 5: Atomic Orbital Basis Sets

Orbital hybridization is a central idea in general chemistry, and is put into practice in computational chemistry. To put this into practice, we need to actually define the atomic orbitals! This module shows how to use Gaussian-type atomic orbitals. 

In [ ]:
import math
import numpy
import matplotlib.pyplot as plt
from pyscf import gto,scf,dft,cc
from pyscf.tools import cubegen

### Example 1: Hydrogen atom 1s orbital in different AO basis sets

The exact nonrelativistic 1s  orbital for a one-electron atom of nuclear charge Ze is, in atomic units, (2Z/n)^(3/2)exp[-Zr/n] This function compares the exact orbital to calculations in various standard AO basis sets

In [ ]:
def psi1s(rs,Z=1,n=1):
    return(((Z/n)**3/(math.pi))**0.5*numpy.exp(-Z*rs/n))
rs=numpy.array(range(60))*0.1
xs=numpy.zeros_like(rs)
coords=numpy.transpose(numpy.array((xs,xs,rs)))
psiexact=psi1s(rs)


basissets=['STO-3G','3-21G','6-31G','def2SVP','cc-pVDZ','def2TZVP','cc-pVTZ','aug-cc-pVTZ','def2QZVP','cc-pVQZ','aug-cc-pVQZ','cc-pV5Z']

# Create dictionaries of ground-state energy and wavefunction values
Es={}
psis={'Exact':psiexact}
Ns={}

# Do the calculation in each basis set and save the results 
for b in basissets:
    m=gto.Mole(atom='H',spin=1,basis=b)
    m.build()
    mf=scf.UHF(m)
    mf.kernel()
    aos=m.eval_gto("GTOval_sph",coords)
    mo=numpy.einsum('m,rm->r',mf.mo_coeff[0,:,0],aos)
    Es[b]=mf.e_tot
    psis[b]=mo
    Ns[b]=m.nao

In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=[10,4])
rs2=rs*rs
#axs.set_box_aspect(1)
axs[0].set(xlabel='Distance r from nucleus (bohr)',ylabel='Wavefunction (bohr^(-1/2)')
axs[1].set(xlabel='Distance r from nucleus (bohr)',ylabel='r^2 Wavefunction (bohr^(-1/2)')
for tag in Es.keys():
    axs[0].plot(rs,psis[tag],label=tag)
    axs[1].plot(rs,rs2*psis[tag],label=tag)
axs[0].legend(loc='upper right', edgecolor = 'black', handlelength = 1.2)
plt.show()

Plot errors in the total energy, using a log scale. 

In [ ]:
fig = plt.figure(figsize = (10, 5))
xs = list(Es.keys())
ys = numpy.array(list(Es.values()))+0.5
#ys=numpy.log10(ys+0.5)
plt.bar(xs, ys, color ='green', width = 0.4)
plt.xlabel("Basis set")
plt.ylabel("Total energy error (Hartree)")
ax=plt.gca()
#ax.set_ylim([-0.46,-0.51])
ax.set_yscale('log')
plt.show()

## Example 2: Oxygen IP and fluorine EA in various basis sets 

These are so-called "delta-SCF" calculations, where we explicitly compute the energies of the neutral atom and the anion/cation. This is  generally more accurate than using the neutral atom HOMO/LUMO energies to approximate IP/EA. 

In [ ]:
def ipea(basis):
    a2eV=27.211 # Conversion from Hartree to electron-volts 
    m=gto.Mole(atom='O',basis=basis,charge=0,spin=2)
    m.build()
    fo=scf.UHF(m)
    fo.kernel()
    m=gto.Mole(atom='O',basis=basis,charge=1,spin=3)
    m.build()
    fop=scf.UHF(m)
    fop.kernel()
    
    m=gto.Mole(atom='F',basis=basis,charge=0,spin=1)
    m.build()
    fl=scf.UHF(m)
    fl.kernel()
    m=gto.Mole(atom='F',basis=basis,charge=-1,spin=0)
    m.build()
    flm=scf.UHF(m)
    flm.kernel()
    return[basis,a2eV*(fop.e_tot-fo.e_tot),a2eV*(fl.e_tot-flm.e_tot)]

In [ ]:
dat=[]
for b in ('STO-3G','3-21G','def2TZVP','def2QZVP','aug-cc-pVQZ'):
    dat.append(ipea(b))
dat=list(map(list, zip(*dat)))

In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=[10,5])
width=0.25
xs=numpy.arange(len(dat[0]))+0.5
axs[0].bar(xs,dat[1],width=width,label='Oxygen IP')
axs[1].bar(xs+0.3,dat[2],width=width,label='Fluorine EA')
axs[0].set_title('Oxygen atom IP')
axs[1].set_title('Fluorine atom EA')
axs[0].set_ylabel('Energy (eV)')
axs[0].set_ylim(9,13)
axs[0].set_xticks(xs + width)
axs[0].set_xticklabels(dat[0])
axs[1].set_ylabel('Energy (eV)')
axs[1].set_xticks(xs + width)
axs[1].set_xticklabels(dat[0])
plt.axhline(y=0, color='black',linestyle='-')
plt.show()

The experimental oxygen atom ionization potential is 13.61 eV. See the NIST atomic spectra database https://physics.nist.gov/cgi-bin/ASD/ie.pl?spectra=O&units=1&at_num_out=on&el_name_out=on&seq_out=on&shells_out=on&level_out=on&e_out=0&unc_out=on&biblio=on The calculations give around 12 eV at the basis set limit. Beyond-mean-field approximations will give more accurate IP. 

The experimental fluorine atom electron affinity is 3.41 eV https://webbook.nist.gov/cgi/cbook.cgi?ID=C14762948  The calculations with small basis sets give *negative* electron affinities, indicating that fluorine anion is predicted to be unstable at this level of theory. Large basis sets are essential for computing EA. Beyond-mean-field approximations will give more accurate EA. 


## Example 3: Helium atom static dipole polarizability 

This is a "finite field" calculation, explicitly treating the energy in an applied electric field. The method is adapted from  
https://github.com/pyscf/pyscf.github.io/blob/master/examples/scf/40-apply_electric_field.py

The polarizability is minus the derivative of the dipole moment with respect to applied field, or minus the second derivative of energy with respect to applied field. High accuracy calculations give a polarizability 1.38319 ( e^2 bohr^2/Hartree)  see https://pubs.rsc.org/en/content/articlelanding/1968/sf/sf9680200041. 


In [ ]:
def Hepolar(basis,Efield=0.01):
    mol=gto.Mole(atom='He',basis=basis)
    mol.build()
    mf=scf.RHF(mol)
    mf.kernel()
    E0=mf.e_tot
    P=mf.make_rdm1()
    def apply_field(E):
        mol.set_common_orig([0, 0, 0])  # The gauge origin for dipole integral
        h =(mol.intor('cint1e_kin_sph') + mol.intor('cint1e_nuc_sph')
          + numpy.einsum('x,xij->ij', E, mol.intor('cint1e_r_sph', comp=3)))
        mf = scf.RHF(mol)
        mf.get_hcore = lambda *args: h
        mf.kernel(dm0=P)
        return(mf.e_tot)
    Ep=apply_field([1.0*Efield,0,0])
    Em=apply_field([-1.0*Efield,0,0])
    polar=-(Ep+Em-2*E0)/Efield**2
    return[basis,polar]

In [ ]:
dat=[]
for b in ('STO-3G','6-31G','6-31G(d)','6-31G(d,p)','def2TZVP','def2QZVP'):
    dat.append(Hepolar(b))
dat.append(['Accurate',1.38319])
dat=list(map(list, zip(*dat)))

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=[10,6])
width=0.7
xs=numpy.arange(len(dat[0]))
ax.bar(xs,dat[1],width=width,label='Polarizability (au)')
ax.set_title('Helium atom polarizability')
ax.set_ylabel('Polarizability (au)')
ax.set_xticks(xs )
ax.set_xticklabels(dat[0])
plt.show()

The first three basis sets do not include polarization functions (p-type atomic orbitals) on helium. These give a predicted polarizability of zero! Polarization functions are necessary to give a physically meaningful predicted polarizability.

## Example 4: CO heat of formation, basis set convergence, DFT vs coupled cluster 

We compute the heat of formation 

C(standard) + O(standard) --> CO(g) 

by computing the reaction energy 
C(g)+O(g)-->CO(g) 

and using experimental values for the atomic heats of formation 
C(standard)-->C(g)
O(standard)-->O(g) 

Experimentally, the NIST Chemistry WebBook gives standard heats of formation (kJ/mol) of 

C(g): 716.68
O(g): 249.18 
CO(g): -110.53

In [ ]:
a2kJ=2625.5
dhC=716.68
dhO=249.18

def CODH(basis):
    xc='b3lyp'
    
    mc=gto.Mole(atom='C',basis=basis,spin=2)
    mc.build()
    mdc=dft.UKS(mc,xc=xc)
    mdc.kernel()
    mfc=scf.UHF(mc)
    mfc.kernel()
    mxc=cc.CCSD(mfc)
    mxc.kernel()
    ecc=mxc.e_tot+mxc.ccsd_t()
    
    mo=gto.Mole(atom='O',basis=basis,spin=2)
    mo.build()
    mdo=dft.UKS(mo,xc=xc)
    mdo.kernel()
    mfo=scf.UHF(mo)
    mfo.kernel()
    mxo=cc.CCSD(mfo)
    mxo.kernel()
    eco=mxo.e_tot+mxo.ccsd_t()
    
    mco=gto.Mole(atom='C 0.0 0.0 0.0; O 1.128 0.0 0.0',basis=basis,spin=0)
    mco.build()
    mdco=dft.UKS(mco,xc=xc)
    mdco.kernel()
    mfco=scf.UHF(mco)
    mfco.kernel()
    mxco=cc.CCSD(mfco)
    mxco.kernel()
    ecco=mxco.e_tot+mxco.ccsd_t()
    
    dh1=a2kJ*(mdco.e_tot-mdo.e_tot-mdc.e_tot)  +dhC+dhO
    dh2=a2kJ*(ecco-ecc-eco)  +dhC+dhO  
    return[basis,dh1,dh2]

In [ ]:
dat=[]
for b in ('3-21G','6-31G(d)','def2TZVP','def2QZVP'):
    dat.append(CODH(b))
dat=list(map(list, zip(*dat)))

In [ ]:
dat=list(map(list, zip(*dat)))
dat.append(['Expt',-110.53,-110.53])
dat=list(map(list, zip(*dat)))

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=[10,6])
width=0.2
xs=numpy.arange(len(dat[0]))
ax.bar(xs,dat[1],width=width,label='B3LYP')
ax.bar(xs+width,dat[2],width=width,label='CCSD(T)')
ax.set_title('CO heat of formation')
ax.set_ylabel('Heat of formation (kcal/mol)')
ax.set_xticks(xs)
ax.set_xticklabels(dat[0])
plt.axhline(y=0, color='black',linestyle='-')
plt.show()

Your assignment for this module has two parts

Part 1: Compute the static dipole polarizability of H2, comparing logitudinal polarizability along the bond axis, to transferse polarizability orthogonal to the bond axis. Use the same basis sets as above. Explain why the longitudinal vs. transverse polarizabilities have different basis set dependence. 

Part 2: Compute the heat of formation of H2, following the procedure used above for carbon monoxide. Which results better match experiment? 